2018-2-5
--
此篇文章结合[Python Algorithms](https://www.goodreads.com/book/show/9537756-python-algorithms) 第八章和[Leetcode](https://leetcode.com/)上动态规划相关题目．为笔者动态规划编程思想学习初步小结．

## 1. 概念初解 

动态规划的思想和分治法类似，都是将问题分解成子问题．所不同的是动态规划中子问题直接有重叠，而分治法的子问题直接没有重叠．

动态规划实际上是将递归(recursion)反过来，转化为一个迭代(iteration)的过程，同时在这个过程中还会逐步的填满(比如某个多维数组)．实际上是一个**空间换时间**的算法．

几种方法之间的关系：
--
Brute force(有助于理解问题) $\longrightarrow$ Recursion(一些因素限制：limited stack depth; function call overhead) $\longrightarrow$ Recursion with memorization $\longrightarrow$ Danamic programming 

涉及到几种重要思想：
--
- Induction
- Relaxiation
- Strong assumption

## 2. 适用情况 

1. 最有子结构性质：问题的最优解包含的子问题的解也是最优的
1. 无后效性：子问题的接一旦确定，便不受在其后包含它的更大问题的求解影响
1. 子问题重叠性质

## 3. 思路 

归纳法(Induction)的思想在动态规划中很重要．我们可以通过归纳法得出算法的递归办解决办法，然后逆过来转化为一个迭代版本.

动态规划：
1. Recursion with memorization

    1. Start 1st, next step
    1. Start last, previous
1. Iteration with relaxation

    1. relax edges out 
    1. relax deges in 

## 3. 实例 

Longest Common Subsequence(LCS)问题：
--


In [1]:
# Memoizing decorator 
from functools import wraps
from functools import wraps
def memo(func):
    cache = {}
#     @wraps(func)
    def wraps(*arg):
        if arg not in cache:
            cache[arg] = func(*arg)
        return cache[arg]
    return wraps

In [2]:
lst = [1, 0, 7, 2, 8, 3, 4, 9]

In [28]:
# recursive version
def rec_lis(lst):
    @memo
    def L(cur):
        res = 1
        for pre in range(cur):
            if lst[pre] <= lst[cur]:
                res = max(res, 1 + L(pre))
        return res
    return max(L(i) for i in range(len(lst)))
#     return [L(i) for i in range(len(lst))]

# iterative version
def basic_lis(lst):
    L = [1]*len(lst)
    for cur, val in enumerate(lst):
        for pre in range(cur):
#             print('l0', L[0])
            if lst[pre] <= val:
                L[cur] = max(L[cur], 1 + L[pre])
#     return L
    return max(L)

In [29]:
rec_lis(lst)

5

In [30]:
basic_lis(lst)

5

## leet code 383. Counting bits

In [129]:
# brute forece  
#2.93%
def naive(num):
    def count(i):
        res = []
        while i != 0:
            res.append(i%2)
            i = i//2
        return sum(res)
    re = []
    for i in range(num+1):
        re.append(count(i))
    return re



# Recursive version  L(i) = L(i//2) + i%2  
# 2.11%
def recur(num):
    dp = [0]*(num+1)
    def L(i):
        if i == 0:
            dp[i] = 0
            return dp[i]
#         print('i', i)
#         print('i//2', i//2)
#         print('i%2', i%2)
        dp[i] = L(i//2) + i%2
        return dp[i]
    for i in range(num+1):
        L(i)
    return dp 

# Recursive version with memorization
# 4.57%
from functools import wraps
def memo(func):
    cache = {}
#     @wraps(func)
    def wraps(*arg):
        if arg not in cache:
            cache[arg] = func(*arg)
        return cache[arg]
    return wraps
def recur_memo(num):
    dp = [0]*(num+1)
    @memo
    def L(i):
        if i == 0:
            dp[i] = 0
            return dp[i]
#         print('i', i)
#         print('i//2', i//2)
#         print('i%2', i%2)
        dp[i] = L(i//2) + i%2
        return dp[i]
    for i in range(num+1):
        L(i)
    return dp 


# DP version
# strong assumption dp[i] = dp[i//1] + i%2
% 31.03
def dp(nums):
    dp = [0]*(nums+1)
    for idx in range(1, nums+1):
        dp[idx] = dp[idx//2] + idx%2
    return dp    

In [130]:
naive(5)

[0, 1, 1, 2, 1, 2]

In [131]:
recur(5)

[0, 1, 1, 2, 1, 2]

In [132]:
recur_memo(5)

[0, 1, 1, 2, 1, 2]

In [101]:
dp(5)

[0, 1, 1, 2, 1, 2]

## Leetcode 712. Minimum ASCII delete sum for two strings 

In [133]:
s1 = 'delete'
s2 = 'leet'

Induction process:
--
L(i, j) =
1. ord(s1[0:i+1])  if j=0
1. ord(s2[0:j+1])  if i=0
1. min(L(i-1, j)+ord(s1[i]), L(i, j-1) + ord(s2[j]))

In [2]:
# recursive version

def rec(s1, s2):
    def L(i, j):
        if i == 0:
            return sum(ord(s2[idx]) for idx in range(j+1))
        elif j == 0:
            return sum(ord(s1[idx]) for idx in range(i+1))
        if s1[i] == s2[j]:
            return L(i-1, j-1)
        else:
            return min(L(i-1, j)+ord(s1[i]), L(i, j-1) + ord(s2[j]))
    return L(len(s1)-1, len(s2)-1)

# DP version

def dp(s1, s2):
    m = len(s1)
    n = len(s2)
    dp = [[0]*n for _ in xrange(m)]
    for i in xrange(n):
        for j in xrange(m):
            if i == 0:
                dp[i][j] = sum(ord(s2[idx]) for idx in range(j+1))
            elif j == 0:
                dp[i][j] = sum(ord(s1[idx]) for idx in range(i+1))
            if s1[i] == s1[j]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = min(dp[i-1][j] + ord(s1[i]), dp[i][j-1] + ord(s2[j]))
    return dp
    